### Dynamically route logic based on input

In [2]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain.invoke({"question": "how do I call Anthropic?"})

'Anthropic'

In [4]:
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)
anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)

In [5]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)

In [ ]:
# 1. run chain and get output from chain 
# 2. pass to branch
# 3. run branch 

full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch
full_chain

In [9]:
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content="As Dario Amodei told me, using Anthropic involves understanding and applying the principles of anthropic reasoning. This includes considering the self-sampling assumption and the self-indication assumption, which are key components of anthropic thinking. To use Anthropic effectively, you need to carefully consider the observer selection effects and account for them in your reasoning. Additionally, it's important to keep in mind the various anthropic frameworks and theories, such as the Doomsday argument or the Sleeping Beauty problem, which can help guide your analysis in different scenarios. Ultimately, using Anthropic requires a deep understanding of probability theory, decision theory, and philosophy, as well as a willingness to grapple with complex and counterintuitive concepts.")

In [10]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, to use LangChain, you first need to download and install the LangChain application on your device. Once installed, open the app and create an account by providing the necessary information. After creating an account, you can log in and access the main interface of LangChain.\n\nTo translate text using LangChain, simply enter or paste the text you want to translate into the designated area. Select the source language from the drop-down menu and choose the target language you want to translate the text into. Then, click on the "Translate" button to generate the translation.\n\nLangChain also offers advanced features such as voice translation and image translation. To utilize these features, you can either speak into the microphone or take a picture using your device\'s camera, and LangChain will provide translations accordingly.\n\nAdditionally, LangChain allows you to save and organize your translated texts, create language learning flashcar

In [11]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='2 + 2 equals 4.')

### Using a custom function

In [14]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [15]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [16]:
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content="As Dario Amodei told me, using Anthropic involves several steps. First, you need to define the problem you want to solve and identify the relevant variables and constraints. Then, you can use Anthropic's tools and techniques, such as probabilistic programming and Bayesian inference, to model and analyze the problem. This may include running simulations, optimizing parameters, or making predictions. Finally, you can evaluate the results and iterate on your models and analyses to refine your understanding of the problem.")